<a href="https://colab.research.google.com/github/ozloty06/PySpark_Amazon-Vine-Analysis/blob/main/Practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [3]:
# 16.4.3
# Start Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameFunctions").getOrCreate()

# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/dataviz-curriculum/day_1/wine.csv"
spark.sparkContext.addFile(url)
df= spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [17]:
# Order a DataFrame by ascending values 
df.orderBy(df["points"].desc()).show(5)

+-------+--------------------+--------------------+------+-----+----------+--------------------+---------------+---------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|       region_2|        variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+---------------+---------------+--------------------+
|  Italy|Here's a ���wow�۝...|            Messorio|    99|  320|   Tuscany|             Toscana|           null|         Merlot|        Le Macchiole|
|     US|A stupendous Pino...|Precious Mountain...|    99|   94|California|        Sonoma Coast|         Sonoma|     Pinot Noir|     Williams Selyem|
|  Italy|The 2007 Ornellai...|           Ornellaia|    99|  200|   Tuscany|  Bolgheri Superiore|           null|      Red Blend|Tenuta dell'Ornel...|
|     US|Depth and texture...|Royal City Stoner...|    99|  140|Washington|Columbia Valley (WA)|Colu

In [19]:
# Order a DataFrame by ascending values 
df.orderBy(df["price"].desc()).show(50)

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+-----------------+--------------------+--------------------+
|             country|         description|         designation|              points|               price|        province|            region_1|         region_2|             variety|              winery|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+-----------------+--------------------+--------------------+
|               Italy|"Made with a blen...| black pepper and...| biodynamic and o...|       Munjebel 7 VA|              91|                  50|Sicily & Sardinia|                Etna|                null|
|              France|"The nose is domi...| and at this poin...|   given the price."|         Les Charmes|              87|                  65|         Burgundy|   Chambolle-Musig

In [20]:
# Import function
from pyspark.sql.functions import avg
df.select(avg("points")).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [21]:
# Filter by price on certain columns
df.filter("price<20").select(['points','country','winery','price']).show(5)

+------+--------+--------------------+-----+
|points| country|              winery|price|
+------+--------+--------------------+-----+
|    90|Bulgaria|        Villa Melnik|   15|
|    90|   Spain|      Don Bernardino|   17|
|    90|      US|            De Loach|   18|
|    91|      US|   Trinity Vineyards|   19|
|    91|Portugal|Adega Cooperativa...|   15|
+------+--------+--------------------+-----+
only showing top 5 rows



In [23]:
df.filter(df["country"] == "US").show(50)

+-------+--------------------+--------------------+------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|            region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|                Napa|  Cabernet Sauvignon|               Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|              Sonoma|     Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|    Oregon|   Willamette Valley|   Willamette Valley|          Pinot Noir|               Ponzi|
|     US|This re-named vin...|          

In [30]:
df.filter(df["price"] < 45).show(50)

+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
|  country|         description|         designation|points|price|        province|            region_1|            region_2|             variety|              winery|
+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
|       US|Heitz has made th...|          Grignolino|    95|   24|      California|         Napa Valley|                Napa|                Ros̩|               Heitz|
| Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|        Bulgaria|                null|                null|              Mavrud|        Villa Melnik|
|       US|Steely and perfum...|            Babushka|    90|   37|      California|Russian River Valley|              Sonoma|          Chardonnay|            Ze

In [38]:
# filter on two conditions
df.filter((df["province"] == "California")&(df["price"] > 200)).show(50)

+-------+--------------------+--------------------+------+-----+----------+--------------------+-------------+--------------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|     region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+-------------+--------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|         Napa|  Cabernet Sauvignon|               Heitz|
|     US|This blockbuster,...|     Rainin Vineyard|    95|  325|California|Diamond Mountain ...|         Napa|  Cabernet Sauvignon|                Hall|
|     US|This wine is quit...|Heritage Single V...|    92|  250|California|         Napa Valley|         Napa|  Cabernet Sauvignon|     Guarachi Family|
|     US|A small-productio...|         Double Plus|    93|  250|California|       